# Bokeh dashboard for THOR data

In [4]:
import os, io, random
import string
import pickle
import numpy as np
import pandas as pd
import pylab as plt
import seaborn as sns
from collections import OrderedDict
import datetime as dt
import geopandas as gpd

In [1]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, CustomJS, DatePicker
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook
from bokeh.models import (DataTable, GeoJSONDataSource, ColumnDataSource, HoverTool, renderers,
                          Label, LabelSet, CustomJS, MultiSelect, Dropdown, Div)
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
output_notebook()
import panel as pn
import panel.widgets as pnw
pn.extension()

Loading BokehJS ...

In [12]:
pn.interact(lambda x: x+2, x=2)

Column
    [0] Column
        [0] IntSlider(end=6, name='x', start=-2, value=2)
    [1] Row
        [0] Str(int, name='interactive00221')

In [5]:
s = pd.read_pickle('thor_small.pkl')
x = s[~s.TGTLATDD_DDD_WGS84.isnull()].copy()
countries = ['NORTH VIETNAM','SOUTH VIETNAM','LAOS','THAILAND','CAMBODIA']
x = x[x.TGTCOUNTRY.isin(countries)]

In [6]:
colormap={'NORTH VIETNAM':'brown','SOUTH VIETNAM':'orange','LAOS':'red',
                'CAMBODIA':'green','THAILAND':'blue','UNKNOWN':'gray'}
x.MFUNC_DESC.unique()
import matplotlib
cmap = matplotlib.colormaps.get_cmap('Set1')
#names = cols
#olors = cmap(np.linspace(0, 1, len(names)))
#m={i:c for i, c in zip(names, colors)}

providers = ['CARTODBPOSITRON','STAMEN_TERRAIN','OSM','ESRI_IMAGERY']
cats = ['TGTCOUNTRY','WEAPONTYPE','MFUNC_DESC']

In [13]:
def draw_map(df=None, long=None, lat=None, height=500, colorby='TGTCOUNTRY',
             point_size=5,
              tile_provider='CARTODBPOSITRON'):
    tile_provider = get_provider(tile_provider)
    tools = "pan,wheel_zoom,box_zoom,hover,tap,lasso_select,reset,save"
    sizing_mode='stretch_both'

    # range bounds supplied in web mercator coordinates
    k = 6378137
    pad = 700000
    if lat == None:
        lat = 16
    if long == None:
        long = 108
    x = long * (k * np.pi/180.0)
    y = np.log(np.tan((90 + lat) * np.pi/360.0)) * k
      
    p = figure(x_range=(x-pad, x+pad), y_range=(y-pad, y+pad),
               x_axis_type="mercator", y_axis_type="mercator", tools=tools,
               width=height, height=height, sizing_mode=sizing_mode)
    p.add_tile(tile_provider)
    if df is None:
        return
    df.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in df[colorby]]
    #df['size'] = 10
    source = ColumnDataSource(df)    
    p.circle(x='x', y='y', size=point_size, alpha=0.7, color='color', source=source)#, legend_group=colorby)
    p.toolbar.logo = None    
    p.title.text = "date"
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("TGTCOUNTRY", "@TGTCOUNTRY"),
        ("MSNDATE", "@MSNDATE{%F}"),
        ("TAKEOFFLOCATION", "@TAKEOFFLOCATION"),
        ("WEAPONTYPE", "@WEAPONTYPE"),
        ("MFUNC_DESC", "@MFUNC_DESC")     
    ])
    hover.formatters={'@MSNDATE': 'datetime'}
    return p 

In [14]:
def dashboard():
    cols = list(x.columns)
    colorby='TGTCOUNTRY'
    map_pane=pn.pane.Bokeh(width=700)
    df_pane = pn.pane.DataFrame(width=600,height=600)
    date_picker = pnw.DatePicker(name='Pick Date',width=200)
    from datetime import date  
    date_picker.value=date(1965, 1, 1)    
    date_slider = pnw.DateSlider(name='Date', start=dt.datetime(1965, 1, 1), 
                                 end=dt.datetime(1973, 10, 31), value=dt.datetime(1968, 1, 1))      
    tile_select = pnw.Select(name='tile layer',options=providers,width=200)
    filterby_select = pnw.Select(name='filter by',value='',options=['']+cols[1:4],width=200)
    value_select = pnw.Select(name='value',value='',options=[],width=200)
    find_btn = pnw.Button(name='find in region',button_type='primary',width=200)

    def update_tile(event=None):
        p = map_pane.object
        p.renderers = [x for x in p.renderers if not str(x).startswith('TileRenderer')]
        rend = renderers.TileRenderer(tile_source= get_provider(tile_select.value))
        p.renderers.insert(0, rend)

    def update_filter(event):
        col=filterby_select.value
        if col=='':
            value_select.options = []
        else:
            value_select.options = sorted(list(x[col].dropna().unique()))

    def find_in_region(event):
        #get points in selected map area
        p = map_pane.object
        source = p.renderers[1].data_source
        d = x[(x.x>p.x_range.start) & (x.x<p.x_range.end) & (x.y>p.y_range.start) & (x.y<p.y_range.end)]
        #add any filter
        d = do_filter(d)
        if len(d)==0:
            return
        elif len(d)>25000:           
            p.title.text = 'too many points!'
        else: 
            d.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in d[colorby]]        
            source.data = dict(d)
            p.title.text = 'selected %s points' %len(d)
        map_pane.param.trigger('object')     
        return

    def do_filter(d):
        col = filterby_select.value
        val = value_select.value
        if col != '':
            d = d[d[col]==val] 
        return d
    
    def update_date(event):
        date_slider.value = date_picker.value     
        
    def update_map(event=None, date=None):
        p = map_pane.object
        source = p.renderers[1].data_source
        if date == None:
            date = str(date_slider.value)
        d = x[x.MSNDATE==date]
        d = do_filter(d)
        if len(d)==0:
            return  
        d.loc[:,'color'] = [colormap[i] if i in colormap else 'gray' for i in d[colorby]]
        source.data = dict(d)
        p.title.text = date

app=dashboard()

In [15]:
app